In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

## LangChain primeiros passos

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [4]:
output = llm('explique como funções python funcionam')

In [5]:
print(output)



Em Python, uma função é um bloco de código que realiza uma tarefa específica. Uma função começa com a palavra-chave def, seguida pelo nome da função e parênteses. Os parâmetros (argumentos) são opcionais e são passados entre os parênteses. O corpo da função é identado e contém o código que é executado quando a função é chamada. O corpo da função pode conter declarações, instruções e outras funções. Quando a função é chamada, os argumentos fornecidos são passados para a função e ela executa o código no corpo da função. Uma função também pode retornar um valor ou um objeto usando a palavra-chave return. Se uma função não retornar nada, ela implícita retorna o valor None.


In [6]:
print(llm.get_num_tokens('explique como funções python funcionam'))

13


In [7]:
output = llm.generate(['... é a capital da França',
                       'qual é a formula da área de um círculo?'])

In [8]:
print(output.generations)

[[Generation(text='\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nA área de um círculo é dada pela fórmula A = π * r², onde A representa a área e r é o raio do círculo.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [9]:
output.generations[0][0].text

'\n\nParis.'

In [10]:
output = llm.generate(['escreva um slogan original para um restaurante de sushi'] * 3)

In [11]:
for o in output.generations:
    print(o[0].text, end='')



"Sabor de Oriente: Sushi como nunca experimentou!"

"Sushi com sabor de tradição - Delícia no seu prato!"

"Uma experiência única de sabores: Sushi Time!"

## ChatModels: GPT-3.5-Turbo e GPT-4

In [12]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

In [13]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='Você é um especialista em machine learning que responde tudo em português.'),
    HumanMessage(content='explique em um parágrafo o que é machine learning.')
]
output = chat(messages)

In [14]:
print(output.content)

Machine learning, ou aprendizado de máquina, é uma subárea da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos computacionais capazes de aprender e tomar decisões a partir de dados. Ao invés de serem explicitamente programados, esses algoritmos são treinados com exemplos e dados relevantes, permitindo que eles identifiquem padrões, façam previsões e tomem decisões autônomas. O objetivo do machine learning é capacitar as máquinas a aprenderem e melhorarem seu desempenho ao longo do tempo, sem a necessidade de intervenção humana constante.


## Prompt Templates

In [15]:
from langchain.prompts import PromptTemplate

In [16]:
template = """Você é um virologista experiente.
Escreva algumas frases sobre o sequinte {virus} em {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=template
)
print(prompt)

input_variables=['idioma', 'virus'] template='Você é um virologista experiente.\nEscreva algumas frases sobre o sequinte {virus} em {idioma}.'


In [17]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', idioma='Inglês'))
print(output)



1. HIV is a complex virus that continues to challenge the scientific community.
2. HIV infection can be prevented through safe sexual practices and needle exchange programs.
3. Antiretroviral therapy is the standard of care for HIV infection.
4. Early detection of HIV is essential to successful treatment.
5. HIV is a global health issue that requires collaborative efforts.


## Simple Chains

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = """Você é um virologista experiente.
Escreva um resumo sobre o sequinte {virus} em {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HIV', 'idioma': 'english'})

In [19]:
output

'HIV, or Human Immunodeficiency Virus, is a retrovirus that attacks the immune system, specifically the CD4 cells (also known as T cells) which play a crucial role in fighting off infections. This virus is primarily transmitted through sexual contact, blood transfusion, sharing needles, or from an infected mother to her child during childbirth or breastfeeding.\n\nOnce HIV enters the body, it replicates rapidly and progressively weakens the immune system. As a result, individuals infected with HIV become more susceptible to opportunistic infections and certain types of cancers. Without treatment, HIV can progress to AIDS (Acquired Immunodeficiency Syndrome), the most advanced stage of the infection.\n\nThe symptoms of HIV can vary from person to person, and some individuals may experience flu-like symptoms shortly after infection. However, in many cases, individuals may not show any symptoms for several years. This is why it is crucial to get tested for HIV if there has been any potent

## Sequential Chains

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['conceito'],
    template="""Você é um cientista experiente e programador Python.
    Escreva uma função que implementa o conceito de {conceito}."""
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

#--- segunda chain ----

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template="""Dada a função {function} Python, descreva como funciona da forma mais detalhada possível."""
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run("softmax")



> Entering new SimpleSequentialChain chain...


def softmax(x):
    """Calcula a função softmax para uma lista de números x."""
    # Use exp para obter os exponenciais de cada elemento da lista.
    exp_list = [np.exp(i) for i in x]

    # Calcule a soma dos exponenciais.
    sum_exp_list = sum(exp_list)

    # Calcule a probabilidade softmax.
    softmax_list = [j/sum_exp_list for j in exp_list]

    return softmax_list
A função softmax calcula a probabilidade de cada elemento em uma lista de números através da aplicação de uma função exponencial normalizada. 

Primeiro, a função utiliza a biblioteca numpy (importada como "np") para calcular o exponencial de cada elemento da lista de números "x" utilizando a função np.exp(). Esses exponenciais são armazenados em uma nova lista chamada "exp_list".

Em seguida, a função realiza a soma de todos os elementos da lista "exp_list" utilizando a função sum(). Essa soma é armazenada em uma variável chamada "sum_exp_list". 

Por fim, a função

## LangChain Agents

In [21]:
# exemplo exponenciação

5.1 ** 7.3

146306.05007233328

In [22]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.llms import OpenAI

In [23]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonAstREPLTool(),
    verbose=True
)
agent_executor.run('qual a resposta para 5.1 elevado à potência de 7.3')



> Entering new AgentExecutor chain...
 I need to calculate the power of 5.1 to 7.3
Action: python_repl_ast
Action Input: 5.1 ** 7.3
Observation: 146306.05007233328
Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'

In [24]:
from math import factorial; print(f"{round(factorial(20)**0.5, 4)}")

1559776268.6285


### Splitting e Embedding de texto

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('docs/CLT.txt') as f:
    clt = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10,
    length_function=len
)

In [57]:
chunks = text_splitter.create_documents([clt])

In [58]:
chunks[0]

Document(page_content='CLT E A REFORMA TRABALHISTA: EVOLUÇÃO\n                                     NAS RELAÇÕES DE TRABALHO\n\n                                                                                                Eliezer de Queiroz Noleto1')

In [59]:
chunks[1]

Document(page_content='Ao falarmos sobre a Consolidação das Leis do Trabalho (CLT), somos reme-\n          tidos ao período conhecido como Era Vargas. Editada no ano de 1943, a CLT resultou\n          da coletânea de inúmeras leis esparsas então existentes que disciplinavam aspectos\n          sobre direito individual do trabalho, direito coletivo, fiscalização do trabalho, direito\n          processual do trabalho, além de legislações específicas de determinadas profissões.\n                    Esse era o espírito do trabalho desenvolvido pela comissão constituída\n          para elaborar a CLT, evidenciado na exposição de motivos que acompanhou o\n          texto final do projeto. Segundo esse escrito, a consolidação nada mais é do que “a\n          fase própria da concatenação dos textos e da coordenação dos princípios, quando\n          já se denuncia primeiro o pensamento do sistema depois de haverem sido regu-')

In [60]:
chunks[10]

Document(page_content='majoritário dos legisladores, o Parlamento aprovou a chamada Reforma Tra-\nbalhista (Lei nº 13.467/2017), a modificação mais profunda na legislação traba-\nlhista desde a edição da própria CLT. Entendeu-se que a legislação não havia\nacompanhado a evolução vivenciada nas relações de trabalho. Isso teria se\ndado, por exemplo, em relação às novas formas de trabalho que surgiram com\nas inovações tecnológicas ou em virtude da atuação mais presente das enti-\ndades de classe, condições que teriam abrandado a hipossuficiência do traba-\nlhador. Assim, a intenção manifesta do Poder Legislativo com a aprovação da\nLei nº 13.467/2017 foi a de atualizar a CLT aos novos tempos, promovendo-se\numa série de mudanças, algumas pontuais, outras de ordem mais geral. Cabe,\nnesta oportunidade, destacar as principais modificações.\n       A nova lei incluiu um capítulo no Título II da CLT (Capítulo II-A) para dis-\nciplinar o teletrabalho, conceituado como “a prestação de serviço

In [61]:
chunks[10].page_content

'majoritário dos legisladores, o Parlamento aprovou a chamada Reforma Tra-\nbalhista (Lei nº 13.467/2017), a modificação mais profunda na legislação traba-\nlhista desde a edição da própria CLT. Entendeu-se que a legislação não havia\nacompanhado a evolução vivenciada nas relações de trabalho. Isso teria se\ndado, por exemplo, em relação às novas formas de trabalho que surgiram com\nas inovações tecnológicas ou em virtude da atuação mais presente das enti-\ndades de classe, condições que teriam abrandado a hipossuficiência do traba-\nlhador. Assim, a intenção manifesta do Poder Legislativo com a aprovação da\nLei nº 13.467/2017 foi a de atualizar a CLT aos novos tempos, promovendo-se\numa série de mudanças, algumas pontuais, outras de ordem mais geral. Cabe,\nnesta oportunidade, destacar as principais modificações.\n       A nova lei incluiu um capítulo no Título II da CLT (Capítulo II-A) para dis-\nciplinar o teletrabalho, conceituado como “a prestação de serviços preponde-'

In [62]:
len(chunks)

1167

In [63]:
def embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_tokens / 1000 * 0.0001:.6f}')

In [64]:
embedding_cost(chunks)

Total de tokens: 257483
Custo de Embedding em USD: 0.025748


### Creating embeddings

In [65]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [66]:
vector = embeddings.embed_query(chunks[0].page_content)

In [45]:
#vector

In [67]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

In [68]:
indexes = pinecone.list_indexes()
for i in indexes:
    pinecone.delete_index(i)
    print('Feito!')

Feito!


In [69]:
index_name = 'linuxtips'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    print('Feito!')

Feito!


In [70]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

### Conversando com os dados (similarity search)

In [71]:
query = 'explique a remuneração das férias'
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='tativos da respectiva categoria profissional, e pro-         da remuneração da tarefa na data da concessão\nvidenciará a afixação de aviso nos locais de tra-            das férias. (Parágrafo acrescido pelo Decreto-Lei nº 1.535,\nbalho. (Parágrafo acrescido pelo Decreto-Lei nº 1.535, de   de 13/4/1977)\n13/4/1977)                                                    § 3º Quando o salário for pago por percenta-\nArt. 140. Os empregados contratados há menos                gem, comissão ou viagem, apurar-se-á a média\nde 12 (doze) meses gozarão, na oportunidade,                percebida pelo empregado nos 12 (doze) meses\nférias proporcionais, iniciando-se, então, novo             que precederem à concessão das férias. (Parágrafo\nperíodo aquisitivo. (Artigo com redação dada pelo           acrescido pelo Decreto-Lei nº 1.535, de 13/4/1977)\nDecreto-Lei nº 1.535, de 13/4/1977)                           § 4º A parte do salário paga em utilidades será'), Document(page_c

In [72]:
for r in result:
    print(r.page_content)
    print('-' * 50)

tativos da respectiva categoria profissional, e pro-         da remuneração da tarefa na data da concessão
videnciará a afixação de aviso nos locais de tra-            das férias. (Parágrafo acrescido pelo Decreto-Lei nº 1.535,
balho. (Parágrafo acrescido pelo Decreto-Lei nº 1.535, de   de 13/4/1977)
13/4/1977)                                                    § 3º Quando o salário for pago por percenta-
Art. 140. Os empregados contratados há menos                gem, comissão ou viagem, apurar-se-á a média
de 12 (doze) meses gozarão, na oportunidade,                percebida pelo empregado nos 12 (doze) meses
férias proporcionais, iniciando-se, então, novo             que precederem à concessão das férias. (Parágrafo
período aquisitivo. (Artigo com redação dada pelo           acrescido pelo Decreto-Lei nº 1.535, de 13/4/1977)
Decreto-Lei nº 1.535, de 13/4/1977)                           § 4º A parte do salário paga em utilidades será
--------------------------------------------------

### Gerando respostas com LLM

In [74]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=0.5)

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [76]:
query = 'como funciona o décimo terceiro salário?'
resp = chain.run(query)
print(resp)

O décimo terceiro salário, também conhecido como gratificação de Natal, é um benefício garantido por lei aos trabalhadores brasileiros. Ele consiste no pagamento de uma parcela adicional do salário no final do ano, equivalente a 1/12 avos da remuneração devida ao empregado em cada mês trabalhado no ano.

O décimo terceiro salário deve ser pago em duas parcelas: a primeira deve ser paga até o dia 30 de novembro e corresponde a metade do salário do empregado. Já a segunda parcela deve ser paga até o dia 20 de dezembro e corresponde à outra metade do salário, descontando-se o valor do Imposto de Renda, se houver.

É importante ressaltar que o décimo terceiro salário é um direito de todos os trabalhadores com carteira assinada, inclusive os empregados domésticos. Além disso, o valor do décimo terceiro salário deve ser proporcional ao tempo de serviço prestado durante o ano, ou seja, se o empregado trabalhou apenas alguns meses no ano, ele receberá o décimo terceiro proporcional a esse perí